In [ ]:
!pip install google-genai

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Agent ---

def anonymization_agent(data: str, previous_feedback: str = "") -> str:
    """Agent that identifies and redacts protected health information."""
    print("-> [Agent System] Processing dataset for PHI redaction...")

    if previous_feedback:
        prompt = f"""You are a data compliance assistant. Revise the anonymized data based on the human compliance officer's feedback.
        Original Data: {data}
        Human Feedback: {previous_feedback}

        Return ONLY the revised dataset with all PII/PHI replaced with [REDACTED]."""
    else:
        prompt = f"""You are a data compliance assistant. Anonymize the following patient dataset by replacing all names, dates, phone numbers, and addresses with [REDACTED].
        Original Data: {data}

        Return ONLY the anonymized dataset."""

    response = client.models.generate_content(model=MODEL_ID, contents=prompt)
    return response.text

# --- 2. Define the Human-in-the-Loop Orchestrator ---

def human_in_the_loop_workflow(dataset: str) -> str:
    """
    Orchestrator that implements the human-in-the-loop pattern.
    It pauses execution to wait for human review before proceeding.
    """
    print("Starting Human-in-the-Loop Workflow...\n")

    feedback = ""
    iteration = 1

    while True:
        print(f"--- Iteration {iteration} ---")

        # 1. Agent performs the initial task or attempts a correction
        draft = anonymization_agent(dataset, feedback)

        # 2. Trigger Request to External System: The Checkpoint
        print("\n=== PENDING HUMAN REVIEW ===")
        print(draft)
        print("============================\n")

        # 3. External Messaging System (Console Input) waits for Human Reviewer
        print("-> [External System] Waiting for compliance officer review...")
        approval = input("Approve release? (y/n): ").strip().lower()

        # 4. Process Human Evaluation
        if approval == 'y':
            print("-> ✅ [Human Reviewer] Approved. Releasing dataset.")
            # If approved, send response
            return draft
        else:
            # If not approved, follow pre-defined process to generate a solution
            feedback = input("-> ❌ [Human Reviewer] Rejected. Please provide specific feedback for correction: ")
            print("\nSending evaluation back to the Agent System...\n")
            iteration += 1

# --- Execution ---
if __name__ == "__main__":
    # Example sensitive dataset
    sensitive_patient_data = (
        "Patient Name: John Doe\n"
        "DOB: 10/14/1985\n"
        "Condition: Type 2 Diabetes\n"
        "Notes: Patient visited the clinic on 04/12/2023. Prescribed Metformin. "
        "Call at 555-0198 to follow up."
    )

    # Trigger the workflow
    final_output = human_in_the_loop_workflow(sensitive_patient_data)

    print("\n=== Final Approved Output to Downstream System ===")
    print(final_output)

Starting Human-in-the-Loop Workflow...

--- Iteration 1 ---
-> [Agent System] Processing dataset for PHI redaction...

=== PENDING HUMAN REVIEW ===
Patient Name: [REDACTED]
DOB: [REDACTED]
Condition: Type 2 Diabetes
Notes: Patient visited the clinic on [REDACTED]. Prescribed Metformin. Call at [REDACTED] to follow up.

-> [External System] Waiting for compliance officer review...
Approve release? (y/n): y
-> ✅ [Human Reviewer] Approved. Releasing dataset.

=== Final Approved Output to Downstream System ===
Patient Name: [REDACTED]
DOB: [REDACTED]
Condition: Type 2 Diabetes
Notes: Patient visited the clinic on [REDACTED]. Prescribed Metformin. Call at [REDACTED] to follow up.
